In [1]:
import re

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager
from pprint import pprint

from jieba_postag_converter import get_postag_dict



# TF-IDF

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer

def calc_tfidf(data):
    tfidf = TfidfVectorizer()
    x = tfidf.fit_transform(data)
    df_tfidf = pd.DataFrame(x.toarray(), columns=tfidf.get_feature_names())
    return df_tfidf

In [132]:
def rank_words_by_unalighed_diff_counts(df,corpus='cejc',situation='apology',sen_type='query', diff_type='del', reverse=False, tfidf=False):
    # FOR creating words ranking graph and pos rangling graph
#     cols = {'primitive form':'word count', 'pos':'pos count'}
    df = df[(df['corpus']==corpus)&(df['situation']==situation)&(df['sentence type']==sen_type)&(df['difference type']==diff_type)]
      
    cols = {'primitive form':'word count'}
    for label_col, data_col in cols.items():
        if tfidf == True:
            df = df.assign(tfidf = list(np.zeros(len(df))))
            dirs = ['/nfs/nas-7.1/yamashita/LAB/giza-pp/primitive']
            corpora = ['cejc']
            situations = ['apology']
            method_type = ['rewrited']
            sentence_type = ['query']
            data = []
            for d in dirs:
                for c in corpora:
                    for s in situations:
                        for m in method_type:
                            if m != 'original':
                                c = 'mpdd' if corpus == 'cejc' else 'mpdd'
                            for t in sentence_type:
                                path =f'{d}/{c}/{s}/{m}_{t}'
                                with open(path, 'r', encoding='utf_8_sig')as f:
                                    for i,sen in enumerate(f):
                                        line = sen[:-1]
                                        data.append(line)
            tfidf_df=calc_tfidf(data)
            for index, row in df.iterrows():
                line= row['line']
                prim= row['primitive form']
                try:
                    df.loc[index, 'tfidf'] = tfidf_df.loc[line, prim]
                except:
#                     print(prim)
                        pass
                
            df[data_col] = df.groupby(['corpus', 'situation', 'method', 'sentence type', 'difference type',label_col])['tfidf'].transform(np.sum)
        else:
            df[data_col] = df.groupby(['corpus', 'situation', 'method', 'sentence type', 'difference type',label_col])['index'].transform('count')
        
        df_word = df.drop_duplicates(['corpus', 'situation', 'method', 'sentence type', 'difference type',label_col]).sort_values(by=data_col, ascending=False)
        df_word = df_word.dropna(subset=[label_col])

        # GET data for 1 graph
#         difference_type = ['del','add']
        method_type = ['translated','rewrited']
        data, labels = [], []
        _temp_w, _temp_wv = [], [] 
        for method in method_type:
            _temp_w.append(df_word[df_word['method']==method][label_col])
            _temp_wv.append(df_word[df_word['method']==method][data_col])
        # PUT data into dictionary, COUNT freq., CALCULATE (rewrited - translated) and SORT them
        dic = {}
        for i,(wline,vline) in enumerate(zip(_temp_w,_temp_wv)):
            for j,(w,v) in enumerate(zip(wline,vline)):
                if w in dic:
                    dic[w]['each'][i]=v
                    v = v*-1 if i==0 else v
                    dic[w]['diff']=dic.get(w,dic.get('diff',0))['diff']+v
                else:
                    tmp = [0,0]
                    tmp[i] = v
                    v = v*-1 if i==0 else v
                    dic.setdefault(w,{'each':tmp,'diff':v})
        dic = sorted(dic.items(),key=lambda x:x[1]['diff'],reverse=reverse)
#         pprint(dic)
        # exit()
#         data_w, data_wv = [],[[],[]]
#         for key, values in dic:
#             data_w.append(key)
#             data_wv[0].append(values['each'][0])
#             data_wv[1].append(values['each'][1]) 
        data_w, data_wv = [],[]
        for key, values in dic:
            data_w.append(key)
            data_wv.append(values['diff'])
        labels.append(data_w)
        data.append(data_wv)
        df_ = pd.DataFrame([data_w,data_wv]).T.set_axis(['word','diff count'],axis='columns')
        print(df_)
        return df_

In [133]:
def prim2mrph(df,df_,corpus,situation,sen_type,diff_type,mincnt):
    ranked_words_list = df_[df_['diff count'] >= mincnt]['word'].to_list()
#     print(ranked_words_list)
#     print(len(ranked_words_list))
    df = df[df["corpus"].isin([corpus])&\
            df['situation'].isin([situation])&\
            df["difference type"].isin([diff_type])&\
            df["primitive form"].isin(ranked_words_list)&\
            df['sentence type'].isin([sen_type])]['word']
    word_list = df.to_list()
    return word_list

In [134]:
def get_highdiff_mrphwords(df,corpus,situation,sen_type,diff_type,mincnt,reverse,tfidf):
    if diff_type=='add':
        corpus = 'mpdd' if corpus == 'cejc' else 'cejc'
    df_ = rank_words_by_unalighed_diff_counts(df,corpus,situation,sen_type,diff_type,reverse,tfidf)
    word_list = prim2mrph(df,df_,corpus,situation,sen_type,diff_type,mincnt)
    return word_list

In [135]:
t_path = "analysis_table.csv"
df = pd.read_csv(t_path)

In [136]:
corpus = 'cejc'
situation = 'apology'
sen_type = 'query'
diff_type = 'add'
mincnt = 1

word_list = get_highdiff_mrphwords(df,corpus,situation,sen_type,diff_type,mincnt,reverse=False,tfidf=False)
word_list
# len(word_list)
# df_[df_['diff count'] >= 1]

      word diff count
0       ああ        -48
1       ます        -29
2       いる        -23
3        ぬ        -20
4       する        -18
..     ...        ...
260      お          4
261     もう          4
262     お前          5
263  すみません          5
264  申し訳ない          6

[265 rows x 2 columns]


/home/yamashita/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


['お前',
 'ご',
 '家',
 '用事',
 '遅れて',
 '校長',
 '本当に',
 'お',
 'ちゃんと',
 '申請',
 '休んで',
 '本当に',
 'もう',
 '江',
 'お',
 'た',
 'お前',
 '本当に',
 'すみません',
 'お',
 'あいつ',
 'すみません',
 '外',
 '方',
 '農村',
 'すみません',
 'お',
 'お',
 'ごめんなさい',
 '出身',
 'すみません',
 'ごめんなさい',
 'お前',
 'ご',
 '申し訳ない',
 '本当に',
 '申し訳ない',
 '絶対',
 'お前',
 '申し訳ない',
 '大事な',
 '愛',
 '申し訳ない',
 'ごめんなさい',
 'ごめん',
 '今',
 '子供',
 'ごめんなさい',
 '弱って',
 'る',
 '初めて',
 '強',
 '方',
 '子だから',
 '大事な',
 '他の',
 'つながり',
 '遅れた',
 '文',
 'もう',
 'もう',
 'れて',
 'る',
 'た',
 'いいん',
 '今日',
 '市',
 '名',
 '家',
 'どうしても',
 '立て',
 '会った',
 'もらう',
 'れて',
 'もう',
 '元',
 '補償',
 'まずい',
 '申し訳ない',
 '考えて',
 'た',
 'た',
 '不安に',
 'もう',
 'る',
 '兄さん',
 '傷つける',
 'お',
 '兄さん',
 '一',
 'お前',
 '申し訳ない',
 'ありがとう',
 'ください',
 '今',
 'ちゃんと',
 'すみません',
 'ごめんなさい',
 '方',
 'た',
 'そういう',
 'ごめんなさい',
 'あげ',
 'とき',
 'ただ',
 'お',
 '義姉',
 'せて',
 'る',
 '不安に',
 '悪い',
 '意味',
 'お',
 'ください',
 '元',
 '社',
 '主要な',
 '事件',
 '一',
 'ください',
 '申し訳ない',
 'あげ',
 'る',
 'ごめんなさい',
 'いろんな',
 'ともかく',
 'お',
 'ごめん',
 '遅れて',
 'ください',
 '本当に',
 'ご

# TF-IDF main()

In [137]:
    dirs = ['/nfs/nas-7.1/yamashita/LAB/giza-pp/primitive']
    corpora = ['cejc','mpdd']
    situations = ['apology','request','thanksgiving']
    method_type = ['original','translated','rewrited']
    sentence_type = ['query', 'res']
    corpora = ['cejc']
    situations = ['apology']
    method_type = ['original']
    sentence_type = ['query']
    data = []
    for d in dirs:
        for corpus in corpora:
            for situation in situations:
                for m in method_type:
                    for t in sentence_type:
                        path =f'{d}/{corpus}/{situation}/{m}_{t}'
                        with open(path, 'r', encoding='utf_8_sig')as f:
                            for i,sen in enumerate(f):
                                line = sen[:-1]
                                data.append(line)
tfidf_df=calc_tfidf(data)
tfidf_df

,ああ,あう,あげる,あご,あす,あたし,あっ,あと,あなた,あの,...,順番,頼む,願う,食べる,食事,飲み物,飽きる,駄目だ,黒豆,ｐｄｆ
0,0.0,0.0,0.0,0.0,0.0,0.268836,0.000000,0.000000,0.0,0.253486,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.366714,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
414,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.373143,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
415,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.594443,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
416,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
417,0.0,0.0,0.0,0.0,0.0,0.000000,0.499285,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [138]:
t_path = "analysis_table.csv"
df = pd.read_csv(t_path)

corpus = 'cejc'
situation = 'apology'
sen_type = 'query'
diff_type = 'del'
mincnt = 1

word_list = get_highdiff_mrphwords(df,corpus,situation,sen_type,diff_type,mincnt,reverse=True,tfidf=True)
word_list

    word diff count
0     のだ          0
1      ぬ          0
2     いい          0
3     じゃ          0
4    土曜日          0
..   ...        ...
215   一応          0
216   石井          0
217   ない -0.0137169
218   する  -0.044278
219  ごめん  -0.146287

[220 rows x 2 columns]


[]